In [ ]:
import heapq

def heuristic(a, b):
    """Calculate the Manhattan distance heuristic."""
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def astar(grid, start, goal):
    """Implement the A* algorithm."""
    rows, cols = len(grid), len(grid[0])

    open_set = []
    heapq.heappush(open_set, (0, start))  # (cost, (x, y))
    came_from = {}

    g_score = {start: 0}
    f_score = {start: heuristic(start, goal)}

    while open_set:
        current = heapq.heappop(open_set)[1]

        if current == goal:
            return reconstruct_path(came_from, current)

        for neighbor in get_neighbors(current, rows, cols):
            if grid[neighbor[0]][neighbor[1]] == 1:  # If it's an obstacle
                continue

            tentative_g_score = g_score[current] + 1

            if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = tentative_g_score + heuristic(neighbor, goal)

                if neighbor not in [i[1] for i in open_set]:
                    heapq.heappush(open_set, (f_score[neighbor], neighbor))

    return []  # No path found

def get_neighbors(pos, rows, cols):
    """Get valid neighbors for a position."""
    x, y = pos
    neighbors = []

    for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        new_x, new_y = x + dx, y + dy
        if 0 <= new_x < rows and 0 <= new_y < cols:
            neighbors.append((new_x, new_y))

    return neighbors

def reconstruct_path(came_from, current):
    """Reconstruct the path from the end node to the start node."""
    total_path = [current]
    while current in came_from:
        current = came_from[current]
        total_path.append(current)
    return total_path[::-1]  # Return reversed path

def mark_path_on_grid(grid, path):
    """Mark the path on the grid."""
    for (x, y) in path:
        if (x, y) != (0, 0) and (x, y) != (4, 4):
            grid[x][y] = 'P'
    return grid

# Example grid
grid = [
    [0, 0, 0, 1, 0],
    [1, 0, 1, 1, 0],
    [0, 0, 0, 0, 0],
    [0, 1, 1, 1, 1],
    [0, 0, 0, 0, 0]
]

start = (0, 0)
goal = (4, 4)

# Find the path
path = astar(grid, start, goal)

# Mark the path on the grid
if path:
    grid_with_path = mark_path_on_grid(grid, path)
else:
    grid_with_path = "No path found."

# Output the grid with the path
for row in grid_with_path:
    print(row)

[0, 'P', 0, 1, 0]
[1, 'P', 1, 1, 0]
['P', 'P', 0, 0, 0]
['P', 1, 1, 1, 1]
['P', 'P', 'P', 'P', 0]


Implement Hill-Climbing algorithm

In [ ]:
import random

class QueenBoard:
    def __init__(self, size):
        self.size = size
        self.queens = self.random_state()
        self.heuristic_value = self.calculate_heuristic()

    def random_state(self):
        # Generate a random initial state
        return [random.randint(0, self.size - 1) for _ in range(self.size)]

    def calculate_heuristic(self):
        # Calculate the number of pairs of queens that are attacking each other
        attacks = 0
        for i in range(self.size):
            for j in range(i + 1, self.size):
                if self.queens[i] == self.queens[j] or \
                   abs(self.queens[i] - self.queens[j]) == abs(i - j):
                    attacks += 1
        return attacks

    def get_neighbors(self):
        # Generate all possible neighbor states
        neighbors = []
        for i in range(self.size):
            for j in range(self.size):
                if j != self.queens[i]:  # Change the row of the i-th queen
                    new_state = self.queens[:]
                    new_state[i] = j
                    neighbors.append(QueenBoard(self.size))
                    neighbors[-1].queens = new_state
                    neighbors[-1].heuristic_value = new_state.calculate_heuristic()
        return neighbors

    def hill_climb(self):
        current = self
        while True:
            neighbors = current.get_neighbors()
            if not neighbors:
                break
            # Find the neighbor with the lowest heuristic value
            next_state = min(neighbors, key=lambda x: x.heuristic_value)
            if next_state.heuristic_value >= current.heuristic_value:
                break  # Stop if we are at a local maximum
            current = next_state
        return current

def main():
    board_size = 8
    initial_board = QueenBoard(board_size)
    solution = initial_board.hill_climb()

    if solution.heuristic_value == 0:
        print("Solution found:")
        for row in solution.queens:
            print(' '.join('Q' if col == row else '.' for col in range(board_size)))
    else:
        print("No solution found.")

if __name__ == "__main__":
    main()


 Write a Program to Implement the Mini-Max algorithm for a Tic-Tac-Toe game using
python

In [ ]:
import math

# Constants for players
PLAYER_X = 'X'
PLAYER_O = 'O'
EMPTY = ' '

# Initialize the game board
def create_board():
    return [[EMPTY for _ in range(3)] for _ in range(3)]

# Print the game board
def print_board(board):
    for row in board:
        print('|'.join(row))
        print('-' * 5)

# Check for a win
def check_winner(board):
    # Check rows and columns
    for i in range(3):
        if board[i][0] == board[i][1] == board[i][2] != EMPTY:
            return board[i][0]
        if board[0][i] == board[1][i] == board[2][i] != EMPTY:
            return board[0][i]

    # Check diagonals
    if board[0][0] == board[1][1] == board[2][2] != EMPTY:
        return board[0][0]
    if board[0][2] == board[1][1] == board[2][0] != EMPTY:
        return board[0][2]

    return None  # No winner yet

# Check if the board is full
def is_full(board):
    return all(cell != EMPTY for row in board for cell in row)

# Mini-Max algorithm
def minimax(board, depth, is_maximizing):
    winner = check_winner(board)
    if winner == PLAYER_X:
        return -1  # X is the minimizing player
    elif winner == PLAYER_O:
        return 1  # O is the maximizing player
    elif is_full(board):
        return 0  # Draw

    if is_maximizing:
        best_score = -math.inf
        for i in range(3):
            for j in range(3):
                if board[i][j] == EMPTY:
                    board[i][j] = PLAYER_O
                    score = minimax(board, depth + 1, False)
                    board[i][j] = EMPTY
                    best_score = max(score, best_score)
        return best_score
    else:
        best_score = math.inf
        for i in range(3):
            for j in range(3):
                if board[i][j] == EMPTY:
                    board[i][j] = PLAYER_X
                    score = minimax(board, depth + 1, True)
                    board[i][j] = EMPTY
                    best_score = min(score, best_score)
        return best_score

# Find the best move for the AI
def best_move(board):
    move = (-1, -1)
    best_score = -math.inf
    for i in range(3):
        for j in range(3):
            if board[i][j] == EMPTY:
                board[i][j] = PLAYER_O
                score = minimax(board, 0, False)
                board[i][j] = EMPTY
                if score > best_score:
                    best_score = score
                    move = (i, j)
    return move

# Main game loop
def play_game():
    board = create_board()
    print_board(board)

    while True:
        # Player X turn
        row, col = map(int, input("Enter your move (row and column 0-2): ").split())
        if board[row][col] != EMPTY:
            print("Invalid move. Try again.")
            continue
        board[row][col] = PLAYER_X
        print_board(board)

        if check_winner(board) == PLAYER_X:
            print("Player X wins!")
            break
        if is_full(board):
            print("It's a draw!")
            break

        # Player O (AI) turn
        row, col = best_move(board)
        if (row, col) != (-1, -1):
            board[row][col] = PLAYER_O
            print("Computer plays O:")
            print_board(board)

        if check_winner(board) == PLAYER_O:
            print("Player O wins!")
            break
        if is_full(board):
            print("It's a draw!")
            break

# Run the game
if __name__ == "__main__":
    play_game()


 | | 
-----
 | | 
-----
 | | 
-----
